In [1]:
# import tensorflow as tf

# gpus = tf.config.experimental.list_physical_devices('GPU')

# if gpus:

#     try:

#         for gpu in gpus:

#             tf.config.experimental.set_memory_growth(gpu, True)

#     except RuntimeError as e:

#         print(e)

# # 실행가능한 gpu 목록
# tf.config.list_physical_devices('GPU')
# #실행가능한 cpu, gpu 목록
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [1]:
import os
import joblib
from time import time

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, TimeDistributed
from tensorflow.keras import Input, Model, callbacks
from tensorflow.keras.utils import plot_model as plm
from tensorflow.keras.activations import swish
from keras.callbacks import CSVLogger

from tensorflow.compat.v1.keras.backend import set_session
from scikeras.wrappers import KerasRegressor
from tcn import TCN, tcn_full_summary

#-------------------------------------------------------------------------
# .. Set configure

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config))


#-------------------------------------------------------------------------
# .. Data set

element = 'ALLV'
name_list = "./SHEL/namelist.input"

hp_lr = 0.009
hp_pd = 'same'
hp_ns = 1
hp_dl = [1,2,4,8,16,32,48]
hp_ldl = hp_dl[-1] # last dilation factor to make name of save model
hp_bn = True
hp_nf = 80
hp_dr = 0.07
hp_ks = 6

input_size = 6
output_size = 2
num_fct = 48
batch_size = 8
n_iter_search = 20
num_epoch = 600
dev_stn_id = 47105
#dev_stn_id = 875
tran_data_per = "2101_2104_2201_2204"

exp_name = "CNTL"
csv_outdir = './DAOU/LOSS/' + exp_name + '/'
model_outdir = './DAOU/MODL/' + exp_name + '/'
scalr_outdir = './DAOU/SCAL/' + exp_name + '/'
gifd_outdir = './GIFD/' + exp_name + '/'
log_outdir = './DAOU/LOGF/' + exp_name + '/'


if os.path.exists(csv_outdir) != True: os.makedirs(csv_outdir)
if os.path.exists(model_outdir) != True: os.makedirs(model_outdir)
if os.path.exists(scalr_outdir) != True: os.makedirs(scalr_outdir)
if os.path.exists(gifd_outdir) != True: os.makedirs(gifd_outdir)
if os.path.exists(log_outdir) != True: os.makedirs(log_outdir)


In [2]:
import numpy as np
import pandas as pd
import sys

sys.path.insert(0, '../config/')
from global_params import variable_info
#from config.global_params import variable_info

sys.path.insert(0, './INC/')
from data_split import data_split





# 인풋 준비
nwp_file = f"../DAIO/nwp_data_{dev_stn_id}"
obs_file = f"../DAIO/obs_data_{dev_stn_id}"
nwp_data = np.load(nwp_file)
obs_data = np.load(obs_file)
print("="*50, "load data shape")
print(nwp_data.shape)
print(obs_data.shape)


# train([21.01,04, 22.01,04]) / test([23.01,04]) 분할  
class_split = data_split(nwp_data, obs_data)
train_nwp, test_nwp, train_obs, test_obs = class_split.get_split_data()
print("="*50, "split data shape")
print(train_nwp.shape)
print(train_obs.shape)
print(test_nwp.shape)
print(test_obs.shape)



# 결측제거
missing_nwp_train = set(np.where(np.isnan(train_nwp))[0])
missing_obs_train = set(np.where(np.isnan(train_obs))[0])
missing_all_train = list(missing_nwp_train | missing_obs_train)
print("결측 합계: ", len(missing_all_train))
dm_nwp_train = np.delete(train_nwp, missing_all_train, 0)
dm_obs_train = np.delete(train_obs, missing_all_train, 0)
print("shape of after drop")
print(dm_nwp_train.shape)
print(dm_obs_train.shape)

missing_nwp_test = set(np.where(np.isnan(test_nwp))[0])
missing_obs_test = set(np.where(np.isnan(test_obs))[0])
missing_all_test = list(missing_nwp_test | missing_obs_test)
print("결측 합계: ", len(missing_all_test))
dm_nwp_test = np.delete(test_nwp, missing_all_test, 0)
dm_obs_test = np.delete(test_obs, missing_all_test, 0)
print("shape of after drop")
print(dm_nwp_test.shape)
print(dm_obs_test.shape)


# 변수선택
# for 47105
#sel_var = ['NDNSW_surface', 'UGRD_10m', 'VGRD_10m', 'RH_1_5ma', 'MAXGUST_0m', 'PRMSL_meansealevel']
# for 875
sel_var = ['UGRD_10m', 'VGRD_10m', "TMP_1_5m", 'RH_1_5ma', 'PRMSL_meansealevel', "PRES_surface"]

var_list_dict = list(variable_info.keys())
var_index = [ var_list_dict.index(i) for i in sel_var ]
#print(var_list_dict)
#print(var_index)
sel_dm_nwp_train = dm_nwp_train[:,1::,var_index]
sel_dm_nwp_test = dm_nwp_test[:,1::,var_index]
dm_obs_train = dm_obs_train[:,1::,:]
dm_obs_test = dm_obs_test[:,1::,:]


print("="*50, "drop data shape")
print(sel_dm_nwp_train.shape)
print(dm_obs_train.shape)
print(sel_dm_nwp_test.shape)
print(dm_obs_test.shape)

================================================== load data shape
(868, 49, 20)
(868, 49, 2)
================================================== split data shape
(109, 49, 20)
(109, 49, 2)
(61, 49, 20)
(61, 49, 2)
결측 합계:  1
shape of after drop
(108, 49, 20)
(108, 49, 2)
결측 합계:  5
shape of after drop
(56, 49, 20)
(56, 49, 2)
================================================== drop data shape
(108, 48, 6)
(108, 48, 2)
(56, 48, 6)
(56, 48, 2)


In [3]:
# .. 스케일링 및 데이터 분할
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import sys
sys.path.insert(0, './INC')
from step_sampling_for_date import step_sampling_for_date
from hist_and_kde_for_split import hist_and_kde_for_split, hist_and_kde_for_split_UV

#-------------------------------------------------------------------------
# .. Normalize

output_size = 2

# .. initialaize
tr_b, tr_s, tr_f = sel_dm_nwp_train.shape[0], sel_dm_nwp_train.shape[1], sel_dm_nwp_train.shape[2]      
ts_b, ts_s, ts_f = sel_dm_nwp_test.shape[0], sel_dm_nwp_test.shape[1], sel_dm_nwp_test.shape[2]      

# .. get restorator with obs range
nwp_scaler = MinMaxScaler()   # copy default true
obs_scaler = MinMaxScaler()
nwp_scaler.fit(sel_dm_nwp_train.view().reshape(tr_b*tr_s, tr_f))
obs_scaler.fit(dm_obs_train.view().reshape(tr_b*tr_s, output_size))

# .. feature normalize   ( train seq, feature = test seq, feature )
nor_dm_nwp_train = nwp_scaler.transform(sel_dm_nwp_train.reshape(tr_b*tr_s, tr_f))
nor_dm_nwp_train = nor_dm_nwp_train.reshape(tr_b,tr_s,tr_f)
nor_dm_nwp_test = nwp_scaler.transform(sel_dm_nwp_test.reshape(ts_b*ts_s, ts_f))
nor_dm_nwp_test = nor_dm_nwp_test.reshape(ts_b,ts_s,ts_f)

nor_dm_obs_train = obs_scaler.transform(dm_obs_train.reshape(tr_b*tr_s, output_size))
nor_dm_obs_train = nor_dm_obs_train.reshape(tr_b,tr_s, output_size)
nor_dm_obs_test = obs_scaler.transform(dm_obs_test.reshape(ts_b*ts_s, output_size))
nor_dm_obs_test = nor_dm_obs_test.reshape(ts_b,ts_s, output_size)


print ('---------- Final training data shape')
print(type(nor_dm_nwp_train))
print ('tran nwp : ', nor_dm_nwp_train.shape)
print ('tran obs : ', nor_dm_obs_train.shape)
print ('test nwp : ', nor_dm_nwp_test.shape)
print ('test obs : ', nor_dm_obs_test.shape)




#-------------------------------------------------------------------------
# .. split train --> train/valid for hyper-parameter tuning
#tran_x, tran_y, vald_x, vald_y = data_split_5years(re_nwp_stn, re_obs_stn, tran_rate, random_seed) 
tran_rate = 0.8
nbin = 10
random_seed = 1
tran_x, tran_y, vald_x, vald_y = step_sampling_for_date(nor_dm_nwp_train, nor_dm_obs_train, tran_rate, nbin, random_seed)



#-------------------------------------------------------------------------
# .. Plot tran valid

#var_name = ['NDNSW_surface', 'UGRD_10m', 'VGRD_10m', 'RH_1_5ma', 'MAXGUST_0m', 'PRMSL_meansealevel', "OBS"]
exp_name = "stn875"

#hist_and_kde_for_split_UV(exp_name, tran_data_per, dev_stn_id, gifd_outdir, tran_y, vald_y, nor_dm_obs_test)                      
hist_and_kde_for_split(exp_name, exp_name, tran_data_per, dev_stn_id, gifd_outdir, tran_x, tran_y, vald_x, vald_y)
#hist_and_kde_for_split(var_name, exp_name, tran_data_per, dev_stn_id, gifd_outdir,
#                       tran_x, tran_y, vald_x, vald_y )


---------- Final training data shape
<class 'numpy.ndarray'>
tran nwp :  (108, 48, 6)
tran obs :  (108, 48, 2)
test nwp :  (56, 48, 6)
test obs :  (56, 48, 2)
in split, re_nwp_stn shape:  (108, 48, 6)
tran nwp shape:  (86, 48, 6) tran obs shape:  (86, 48, 2)
vald nwp shape:  (22, 48, 6) vald obs shape:  (22, 48, 2)
dropna_nwp shape: (86, 48, 6)
dropna_obs shape: (86, 48, 2)
calc from numpy
tnwp_x_median: 0.4106244214954994
tobs_x_median: 0.4724702380952381
./GIFD/CNTL/KDE_ecmw_tran_var6_2101_2104_2201_2204_47105
dropna_nwp shape: (22, 48, 6)
dropna_obs shape: (22, 48, 2)
calc from numpy
tnwp_x_median: 0.3939926076780676
tobs_x_median: 0.4540816326530611
./GIFD/CNTL/KDE_ecmw_vald_var6_2101_2104_2201_2204_47105
dropna_nwp shape: (86, 48, 6)
dropna_obs shape: (22, 48, 6)
calc from numpy
tnwp_x_median: 0.4106244214954994
tobs_x_median: 0.3939926076780676
./GIFD/CNTL/KDE_ecmw_tran_vald_nwp_2101_2104_2201_2204_47105
dropna_nwp shape: (86, 48, 2)
dropna_obs shape: (22, 48, 2)
calc from numpy


In [4]:
#=========================================================================
# .. Model configuration
import tensorflow_addons as tfa
                

#-------------------------------------------------------------------------
# .. Set batch for cross-validation

print ('input_size: ', input_size)
print ('batch_size: ', batch_size)
print ('time_lenght: ', num_fct)


#-------------------------------------------------------------------------
# .. Set Model


# .. Define model
#def create_model(dropout_rate=0.15, nb_filters=7, kernel_size=3): 
def model_builder(hp): 

        # set called hyper_params set
        hp_nbfilters = hp.Int('nb_filters', min_value = 50, max_value = 100, step = 5)
        hp_kernel_size = hp.Int('kernel_size', min_value = 3, max_value = 12, step = 3)
        hp_dilation = hp.Int('dilation', min_value = 0, max_value = 4, step = 1)
        #hp_dilation = hp.Choice('dilation', values = [[1,2,4], [1,2,4,8], [1,2,4,8,16], [1,2,4,8,16,32], [1,2,4,8,16,32,48]])

        dilation_list = [[1,2,4], [1,2,4,8], [1,2,4,8,16], [1,2,4,8,16,32], [1,2,4,8,16,32,48]]
        dilation = dilation_list[hp_dilation][:]

        

        print ('================== Model called ========================')
        print ('input_size: ', input_size)
        print ('batch_size: ', batch_size)
        print ('time_lenght: ', num_fct)
        print ('nb_filters: ', hp_nbfilters)
        print ('kernel_size: ', hp_kernel_size)
        print ('dilations: ', dilation)
        dropout_rate = np.round(hp_dr,2)
        print ('dropout_rate: ', dropout_rate)
        
        ## .. clear keras model
        K.clear_session()
        # .. create model
        #i = Input( batch_shape=(batch_size, num_fct, input_size) )
        i = Input( batch_shape=(None, num_fct, input_size) )
        o = TCN(return_sequences=True,
                activation=swish,
                nb_filters=hp_nbfilters,
                padding=hp_pd,
                use_batch_norm = hp_bn,
                nb_stacks=hp_ns,
                dropout_rate=hp_dr,
                kernel_size=hp_kernel_size,
                use_skip_connections=True,
                dilations=dilation
                )(i)
        o = TimeDistributed(Dense(output_size, activation='linear'))(o)
        # .. compile
        adam = optimizers.Adam(learning_rate=hp_lr)
        m= Model(inputs=[i], outputs=[o])
        m.compile(optimizer=adam, loss='mse', metrics=[tfa.metrics.RSquare(name='r_square')])
        #m.compile(optimizer=adam, loss='mse', metrics=[r2_3dim_uv])
        m.summary()
        return m




input_size:  6
batch_size:  8
time_lenght:  48


In [5]:
# pip install keras_tuner
# pip install keras_tuner_cv
#from keras_tuner_cv.outer_cv import OuterCV
#from keras_tuner.tuners import RandomSearch
#from sklearn.model_selection import KFold
import IPython
import keras_tuner as kt


#from keras_tuner_cv.outer_cv import OuterCV
#from sklearn.model_selection import KFold



#-------------------------------------------------------------------------
# .. Set metrics
#tf.keras.metrics.R
#f1 = tfa.metrics.F1Score(num_classes = len(CLASSES), average='macro')

#budget = 9
#exp_name = 'B9'

#-------------------------------------------------------------------------
# .. Use bayes_opt

# tuner = kt.Hyperband(hypermodel = model_builder,
#                     objective = kt.Objective('val_r_square', direction='max'), 
#                     max_epochs = budget,
#                     factor = 3,
#                     #overwrite=True,
#                     directory = './OPTM/',
#                     project_name = exp_name)

tuner = kt.BayesianOptimization(model_builder,
                     objective = kt.Objective('val_r_square', direction='max'),
                     overwrite=True,
                     max_trials=50)


# 작성한 Hypermodel 출력
tuner.search_space_summary()




================== Model called ========================
input_size:  6
batch_size:  8
time_lenght:  48
nb_filters:  50
kernel_size:  3
dilations:  [1, 2, 4]
dropout_rate:  0.07
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 48, 6)]           0         
                                                                 
 tcn (TCN)                   (None, 48, 50)            40250     
                                                                 
 time_distributed (TimeDistr  (None, 48, 2)            102       
 ibuted)                                                         
                                                                 
Total params: 40,352
Trainable params: 39,752
Non-trainable params: 600
_________________________________________________________________
Search space summary
Default search space size: 3
nb_filters (Int)
{'defaul

In [6]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)


start = time()
tuner.search(tran_x, tran_y, epochs = 1000, validation_data = (vald_x, vald_y), callbacks = [ClearTrainingOutput()])
#tuner.search(tran_x, tran_y, epochs = 600, validation_data = (vald_x, vald_y))

print("Hyperband search took %.2f seconds"     " parameter settings. " % ((time() - start)))


Trial 50 Complete [00h 01m 44s]
val_r_square: 0.33813273906707764

Best val_r_square So Far: 0.3725329339504242
Total elapsed time: 01h 34m 34s
INFO:tensorflow:Oracle triggered exit
Hyperband search took 5673.93 seconds parameter settings. 


In [ ]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_hps
print(f"""
The hyperparameter search is complete.
    best_kernel_size= {best_hps.get('kernel_size')} 
    best_nb_filters= {best_hps.get('nb_filters')}
    best_dilations= {best_hps.get('dilation')}
""")

# 혹은 결과 출력
tuner.results_summary()


In [9]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_r_square", direction="max")

Trial 40 summary
Hyperparameters:
nb_filters: 100
kernel_size: 3
dilation: 0
Score: 0.3725329339504242

Trial 16 summary
Hyperparameters:
nb_filters: 100
kernel_size: 3
dilation: 0
Score: 0.36801132559776306

Trial 15 summary
Hyperparameters:
nb_filters: 90
kernel_size: 3
dilation: 0
Score: 0.36554673314094543

Trial 03 summary
Hyperparameters:
nb_filters: 60
kernel_size: 3
dilation: 0
Score: 0.36310628056526184

Trial 35 summary
Hyperparameters:
nb_filters: 100
kernel_size: 3
dilation: 0
Score: 0.361371785402298

Trial 47 summary
Hyperparameters:
nb_filters: 100
kernel_size: 3
dilation: 0
Score: 0.36121582984924316

Trial 25 summary
Hyperparameters:
nb_filters: 55
kernel_size: 3
dilation: 0
Score: 0.35837626457214355

Trial 48 summary
Hyperparameters:
nb_filters: 50
kernel_size: 3
dilation: 0
Score: 0.35758647322654724

Trial 07 summary
Hyperparameters:
nb_filters: 50
